# Setup

Install required packages:

Import required packages:

In [0]:
import h5py
import numpy as np
import tensorflow as tf
import keras
import keras.backend as K
from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import cv2
from tqdm import tqdm

# Google Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, drive
from oauth2client.client import GoogleCredentials
from google.colab import drive

Authenticate to Google Drive to access dataset folder:

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
GoogleDrive(gauth)
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


General parameters and settings:

In [0]:
original_dataset_path = "/content/drive/My Drive/document-classification/datasets/rvl-cdip/mini-dataset-1600-200-200.h5"
cnn_dataset_path = "/content/drive/My Drive/document-classification/datasets/rvl-cdip/cnn-mini-dataset-1600-200-200.h5"
cnn_model_path = "/content/drive/My Drive/document-classification/models/model.json"
cnn_weights_path = "/content/drive/My Drive/document-classification/models/weights.h5"
cnn_weights_checkpoint_path = "/content/drive/My Drive/document-classification/checkpoints/checkpoint_weights.h5"
history_stats_path = "/content/drive/My Drive/document-classification/history/history_stats.csv"

# Flag to generate CNN dataset (set it to False if the dataset has already been generated)
generate_dataset = False
# False: skip training process and load the already learned weights to only evaluate the network
train_network = True
# True: resume the training from a checkpoint; False: start a new training
# This flag is checked only if train_network=True
resume_training = True

# Shape of the images read from the mini-dataset
original_img_shape = (1000, 750)
# Shape of the images in input to the CNN
cnn_image_shape = (500, 375)
# Number of images classes
n_classes = 16

Check if GPU is available:

In [0]:
from tensorflow.python.client import device_lib 
display(device_lib.list_local_devices())
assert tf.config.list_physical_devices('GPU')

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6119263266125199351, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13795399320139286188
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2436519779849582483
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11150726272
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7102891325770624637
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

# Dataset preprocessing


**Function to generate a dataset for the CNN (VGG16) starting from the corresponding original dataset.**

It reads all images from the original dataset, preprocesses them according to the CNN requirements (resize, convert to RGB) and finally saves them into the new dataset.

In [0]:
def generate_dataset_for_cnn(hdf_original_file, hdf_cnn_file, dataset_name):
  # Number of data rows to read from original file, process and write to the CNN file at the same time
  # (To speedup read/write operations)
  batch_size = 64

  # Read datasets from the original HDF5 file
  ds_original_imgs = hdf_original_file[dataset_name]
  ds_original_labels = hdf_original_file[dataset_name + "_labels"]

  n_imgs = ds_original_imgs.len()

  # Create omonymous datasets into the HDF5 file for the CNN
  ds_cnn_imgs = hdf_cnn_file.create_dataset(dataset_name, (n_imgs, cnn_image_shape[0], cnn_image_shape[1], 3),\
                                      dtype="float32", compression="gzip")
  ds_cnn_labels = hdf_cnn_file.create_dataset(dataset_name + "_labels", (n_imgs, n_classes),\
                                    dtype="int8", compression="gzip")
  
  n_batches = n_imgs//batch_size
  for i in tqdm(range(n_batches)):
    # Load data in batches
    start_batch_index = i*batch_size
    end_batch_index = np.min([(i+1)*batch_size, n_imgs])
    original_batch_imgs = ds_original_imgs[start_batch_index:end_batch_index]
    original_batch_labels = ds_original_labels[start_batch_index:end_batch_index]

    cnn_batch_imgs = np.empty((batch_size, cnn_image_shape[0], cnn_image_shape[1], 3), dtype=np.float32)
    # Labels do not need to be processed
    cnn_batch_labels = original_batch_labels

    # Preprocess all images in batch
    for i in range(batch_size):
      original_img = original_batch_imgs[i]
      # De-normalization: bring back pixel values to the [0,255] range
      non_normalized_img = np.asarray(original_img * 255, dtype=np.uint8)
      # Scale the image to the target shape (height/width ratio is preserved due
      # to the original and target shape values)
      scaled_img = cv2.resize(non_normalized_img, (cnn_image_shape[1], cnn_image_shape[0]),\
                              interpolation=cv2.INTER_AREA)
      # Convert grayscale image to RGB (VGG16 needs color images)
      rgb_img = cv2.cvtColor(scaled_img, cv2.COLOR_GRAY2RGB)
      # Normalization
      cnn_batch_imgs[i] = rgb_img / 255
    
    # Write data batches
    ds_cnn_imgs[start_batch_index:end_batch_index] = cnn_batch_imgs
    ds_cnn_labels[start_batch_index:end_batch_index] = cnn_batch_labels



For each set (train. validation, test) in the original HDF5 dataset file, generate a set in a new HDF5 file to be used during the CNN training phase:

In [0]:
if generate_dataset:
  with h5py.File(original_dataset_path, 'r') as hdf_original_file, h5py.File(cnn_dataset_path, 'w') as hdf_cnn_file:
    print("Generating training dataset for CNN...")
    generate_dataset_for_cnn(hdf_original_file, hdf_cnn_file, "train")
    print("Generating validation dataset for CNN...")
    generate_dataset_for_cnn(hdf_original_file, hdf_cnn_file, "valid")
    print("Generating test dataset for CNN...")
    generate_dataset_for_cnn(hdf_original_file, hdf_cnn_file, "test")

#Utility classes

##Data generator definition

Class to handle large datasets (see https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly for further details)

In [0]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, ds_imgs, ds_labels, batch_size=32, dim=(32,32,32), n_channels=3,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.ds_labels = ds_labels
        self.ds_imgs = ds_imgs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.ds_imgs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.ds_imgs.len())
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels), dtype=np.float32)
        y = np.empty((self.batch_size, self.n_classes), dtype=int)

        # Generate data
        for batch_index, ds_index in enumerate(indexes):
            # Store sample
            X[batch_index,:] = self.ds_imgs[ds_index]

            # Store class
            y[batch_index,:] = self.ds_labels[ds_index]

        return X, y

## History recording

Callback class that records into a csv file the loss values and other performance measures at each training epoch

In [0]:
class LossHistory(keras.callbacks.Callback):
    def __init__(self, file_path, resume_training):
        super(LossHistory, self).__init__()
        self.file_path = file_path
        self.resume_training = resume_training


    def on_train_begin(self, logs={}):
        # if we are starting a new training, create the csv file and the header row
        if not self.resume_training:
            header = np.asarray([["Train-Loss", "Train-Accuracy", "Validation-Loss", "Validation-Accuracy"]])
            np.savetxt(self.file_path, header, fmt='%s', delimiter=",")

    def on_epoch_end(self, epoch, logs=None):
        with open(self.file_path, "a") as csv_file:
            statistics = np.asarray([[logs.get("loss"), logs.get("accuracy"),\
                                     logs.get("val_loss"), logs.get("val_accuracy")]])
            np.savetxt(csv_file, statistics)

##Custom metrics

In [0]:
# class MulticlassTruePositives(tf.keras.metrics.Metric):
#     def __init__(self, name='multiclass_true_positives', **kwargs):
#         super(MulticlassTruePositives, self).__init__(name=name, **kwargs)
#         self.hit_per_class = self.add_weight(name='hit_per_class', shape=(1, n_classes) initializer='zeros')
#         self.n_entries_per_class = self.add_weight(name='n_entries_per_class', shape=(1, n_classes) initializer='zeros')

#     def update_state(self, y_true, y_pred, sample_weight=None):
#         class_index = tf.argmax(y_pred, axis=1)
#         hit = tf.cast(y_true, 'int32') == tf.cast(y_pred, 'int32')

#         if hit:
#             tf.convert_to_tensor


#         values = tf.cast(y_true, 'int32') == tf.cast(y_pred, 'int32')
#         values = tf.cast(values, 'float32')
#         if sample_weight is not None:
#             sample_weight = tf.cast(sample_weight, 'float32')
#             values = tf.multiply(values, sample_weight)
#         self.true_positives.assign_add(tf.reduce_sum(values))

#     def result(self):
#         return self.true_positives

#     def reset_states(self):
#         # The state of the metric will be reset at the start of each epoch.
#         self.true_positives.assign(0.)

# CNN model definition

**For the following part refer to:** https://www.pyimagesearch.com/2019/06/24/change-input-shape-dimensions-for-fine-tuning-with-keras/

Use the VGG16 as base model for fine-tuning:

In [0]:
# load VGG16, ensuring the head FC layer sets are left off, while at
# the same time adjusting the size of the input image tensor to the
# network
baseModel = keras.applications.VGG16(weights=None, include_top=False,
	input_tensor=Input(shape=(cnn_image_shape[0], cnn_image_shape[1], 3)), pooling="max")

Add the custom structure (to be trained) on top of the (trained) convolutional part:

In [0]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = Dense(4096, activation="relu")(headModel)
headModel = Dense(4096, activation="relu")(headModel)
headModel = Dense(n_classes, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
cnn_model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
# for layer in baseModel.layers:
# 	layer.trainable = False

Compile the model basing on the configuration flags:

In [0]:
opt = Adam(lr=0.005)

# If we not want to train the network, load the already trained weights
if not train_network:
    cnn_model.load_weights(cnn_weights_path)
# If the training process has to be resumed load the checkpointed weights
elif resume_training:
    cnn_model.load_weights(cnn_weights_checkpoint_path)

cnn_model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

Show model structure info:

In [0]:
print("[INFO] summary for the whole model...")
print(cnn_model.input_shape)
print(cnn_model.output_shape)
print(cnn_model.summary())

[INFO] summary for the whole model...
(None, 500, 375, 3)
(None, 16)
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 500, 375, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 500, 375, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 500, 375, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 250, 187, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 250, 187, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 250, 187, 128)     147584    
________________________________________________________

# Training

Save the network model into a json file and its weights in a separate HDF5 file:

In [0]:
def save_cnn_to_file(cnn):
  # serialize model to JSON
  model_json = cnn.to_json()
  with open(cnn_model_path, 'w') as json_file:
    json_file.write(model_json)
  # serialize weights to HDF5
  cnn.save_weights(cnn_weights_path)
  print("Model saved")

Train the network (if the corresponding flag is set) and save it into the Drive.

Save a model checkpoint after each improvement in accuracy with respect to the validation set.

In [0]:
# Data generators params
data_gen_params = {'dim': (cnn_image_shape[0], cnn_image_shape[1]),
        'batch_size': 32,
        'n_classes': n_classes,
        'n_channels': 3,
        'shuffle': True}

if train_network:
    with h5py.File(cnn_dataset_path, 'r') as hdf_cnn_file:
        # Generators
        training_generator = DataGenerator(hdf_cnn_file["train"], hdf_cnn_file["train_labels"], **data_gen_params)
        validation_generator = DataGenerator(hdf_cnn_file["valid"], hdf_cnn_file["valid_labels"], **data_gen_params)
        
        checkpoint = ModelCheckpoint(cnn_weights_checkpoint_path)#, monitor="val_accuracy", mode="max", save_best_only=True)
        es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=100, restore_best_weights=True)
        loss_history = LossHistory(file_path=history_stats_path,resume_training=resume_training)

        # Train model on dataset
        history = cnn_model.fit_generator(generator=training_generator,
                            validation_data=validation_generator,
                            use_multiprocessing=True, epochs=1000,
                            workers=0, callbacks=[es, checkpoint, loss_history], verbose=1)
        
        print("\nTraining completed")

    save_cnn_file(cnn_model)

Epoch 1/1000
50/50 [==============================] - 642s 13s/step - loss: 2.7727 - accuracy: 0.0756 - val_loss: 2.7666 - val_accuracy: 0.0625
Epoch 2/1000
50/50 [==============================] - 638s 13s/step - loss: 2.7724 - accuracy: 0.0756 - val_loss: 2.7673 - val_accuracy: 0.0625
Epoch 3/1000
44/50 [=========================>....] - ETA: 1:14 - loss: 2.7728 - accuracy: 0.0739

Evaluate the model on the test set:

In [0]:
with h5py.File(cnn_dataset_path, 'r') as hdf_cnn_file:
    # Evaluate the trained model on the test set
    test_generator = DataGenerator(hdf_cnn_file["test"], hdf_cnn_file["test_labels"], **data_gen_params)
    score = cnn_model.evaluate_generator(test_generator, verbose=1)

    print("\nAccuracy on test set: " + str(score))

In [0]:
with h5py.File(cnn_dataset_path, 'r') as hdf_cnn_file:
    ds_imgs = hdf_cnn_file["test"]
    ds_labels = hdf_cnn_file["test_labels"]
    n_test_imgs = ds_imgs.len()

    predicted_labels = cnn_model.predict(x=ds_imgs, batch_size=data_gen_params["batch_size"])

    ds_labels_explicit = np.argmax(ds_labels, axis=-1)
    predicted_labels_explicit = np.argmax(predicted_labels, axis=-1)
    print(ds_labels_explicit.shape)
    print(predicted_labels_explicit.shape)
    comparison_matrix = np.empty((ds_labels.shape[0], 2), dtype=np.int32)
    comparison_matrix[:,0] = predicted_labels_explicit
    comparison_matrix[:,1] = ds_labels_explicit
    print(comparison_matrix.shape)
    print(comparison_matrix)
